In [5]:
import hashlib

# 0.3 Tagged Hashes

Tagged Hashes are cryptographic hash functions that are used throughout the schnorr/taproot specifications. Their purpose is to make sure that hashes used in one context can't be used in another context. This avoids issues like [CVE-2012-2459](https://en.bitcoin.it/wiki/Common_Vulnerabilities_and_Exposures#CVE-2012-2459).

[BIP-schnorr](https://github.com/sipa/bips/blob/bip-schnorr/bip-schnorr.mediawiki) outlines an example concern:

> For example, without tagged hashing, a bip-schnorr signature could also be valid for a signature scheme where the only difference is that the arguments to the hash function are reordered. Worse, if the bip-schnorr nonce derivation function was copied or independently created, then the nonce could be accidentally reused in the other scheme leaking the secret key.

Tagged hash functions are tweaked with a tag name that is dependent on the context. For schnorr and taproot the scheme involves prefixing the data-to-be-hashed with `SHA256(TagName) || SHA256(TagName)`:

**`tagged_hash("TagName")`** = `sha256(sha256("TagName") + sha256("TagName") + data)`

The prefixed, context-dependent tag data lessens, but does not rule out, the feasibility of a hash collision.

The proposal suggests to hash the tag twice because:

> this is a 64-byte long context-specific constant and the SHA256 block size is also 64 bytes, optimized implementations are possible (identical to SHA256 itself, but with a modified initial state). Using SHA256 of the tag name itself is reasonably simple and efficient for implementations that don't choose to use the optimization.

The following code is an implementation of the `tagged_hash` function in python.

In [10]:
def tagged_hash(tag, input_data):
    data = hashlib.sha256(tag.encode('utf-8')).digest()
    data += data
    data += input_data
    return hashlib.sha256(data).digest()

print(tagged_hash("SampleTagName", b'Input data as bytes').hex())

0966a0c836c19e0d3bf22d6cf111186c00a2a42b8f39cea523692d80623a5c2f
